# Neural Networks Sprint Challenge

## 1) Define the following terms:

- Neuron
- Input Layer
- Hidden Layer
- Output Layer
- Activation
- Backpropagation

Neuron: a container that holds a single scalar (usually a floating-point number) called an activation; neurons combine to form columns that make up either the input data or layers in a neural network. Also called 'node'.   
Input Layer: Ryan Allred mentioned during lecture that sometimes 'layer' is a misnomer with regard to input. In any case, this is the left-most information in a NN and is in the form of an array of rows by columns.  
Hidden Layer: this is a true layer, comprised of neurons, of which at least 1 is required to form a non-perceptron NN. Each hidden layer in a NN receives activations from the input or previous hidden layer, applies a weight to each activation then a bias, and forwards the new activations to each neuron in the next hidden layer or output layer.  
Output Layer: the right-most layer in a NN, this receives activations from the previous layer then returns final scalars, either integer or float, that provide an information array about the question of interest, eg, classification or regression.  
Activation: a single scalar found in a neuron.  
Backpropagation: an algorithmic process by which weights in a NN are revised in a backwards propagated fashion after each training epoch, ie last weight is revised, then one just prior, then one before that, etc.

## 2) Create a perceptron class that can model the behavior of an AND gate. You can use the following table as your training data:

| x1 | x2 | x3 | y |
|----|----|----|---|
| 1  | 1  | 1  | 1 |
| 1  | 0  | 1  | 0 |
| 0  | 1  | 1  | 0 |
| 0  | 0  | 1  | 0 |

In [1]:
import numpy as np

# Define x1, x2, x3

x1 = [1, 1, 0, 1]
x2 = [1, 0, 1, 0]
x3 = [1, 1, 1, 1]
y_list = [1, 0, 0, 0]

X = np.array(list(zip(x1, x2, x3, np.ones(4))))
y = np.array([[val] for val in y_list])
X.shape, y.shape

((4, 4), (4, 1))

In [7]:
# Write perceptron class


class ANDPerceptron():
    def __init__(self, X, y, niter=100):
        self.X = X
        self.y = y
        self.niter = niter
       
    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))
    
    def sigmoid_prime(self, x):
        return self.sigmoid(x) * (1 - self.sigmoid(x))
    
    def fit(self):
        # Create weights
        weights = 2 * np.random.random((self.X.shape[0], self.X.shape[1])) - 1

        for iteration in range(self.niter):
            # Weighted sum of inputs and weights
            weighted_sum = np.dot(self.X, weights)

            # Activate with sigmoid function
            activated_output = self.sigmoid(weighted_sum)

            # Calculate Error
            error = self.y - activated_output

            # Calculate weight adjustments with sigmoid_derivative
            adjustments = error * self.sigmoid_prime(activated_output)

            # Update weights
            weights += np.dot(self.X.T, adjustments)
        
        print('optimized weights after training: ')
        print(weights)
        print('\ny:', y)
        print("\noutputs after training:")
        print(activated_output)

In [8]:
AP = ANDPerceptron(X, y)

In [9]:
AP.fit()

optimized weights after training: 
[[ 3.07459478  2.99438472  3.08288513  2.90912249]
 [ 3.86541602  3.78626125  3.88584973  3.72090025]
 [-3.20702151 -1.9927788  -3.59795866 -3.60748039]
 [-2.55054592 -3.64689253 -2.18076543 -1.92065838]]

y: [[1]
 [0]
 [0]
 [0]]

outputs after training:
[[0.76363645 0.75600267 0.7650122  0.74865501]
 [0.0645059  0.0668376  0.06371967 0.06849584]
 [0.13206253 0.13659925 0.13199551 0.14220624]
 [0.0645059  0.0668376  0.06371967 0.06849584]]


## 3) Implement a Neural Network Multilayer Perceptron class that uses backpropagation to update the network's weights. 
- Your network must have one hidden layer. 
- You do not have to update weights via gradient descent. You can use something like the derivative of the sigmoid function to update weights.
- Train your model on the Heart Disease dataset from UCI:

[Github Dataset](https://github.com/ryanleeallred/datasets/blob/master/heart.csv)

[Raw File on Github](https://raw.githubusercontent.com/ryanleeallred/datasets/master/heart.csv)


In [5]:
##### Your Code Here #####

## 4) Implement a Multilayer Perceptron architecture of your choosing using the Keras library. Train your model and report its baseline accuracy. Then hyperparameter tune at least two parameters and report your model's accuracy. 

- Use the Heart Disease Dataset (binary classification)
- Use an appropriate loss function for a binary classification task
- Use an appropriate activation function on the final layer of your network. 
- Train your model using verbose output for ease of grading.
- Use GridSearchCV to hyperparameter tune your model. (for at least two hyperparameters)
- When hyperparameter tuning, show you work by adding code cells for each new experiment. 
- Report the accuracy for each combination of hyperparameters as you test them so that we can easily see which resulted in the highest accuracy.
- You must hyperparameter tune at least 5 parameters in order to get a 3 on this section.

In [6]:
##### Your Code Here #####